<a href="https://colab.research.google.com/github/Maruf346/AI-ML-with-python/blob/main/Dataset_for_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import random

# --- 1. CONFIGURATION ---
NUM_ROWS = 5000
NUM_DEVICES = 30
OUTPUT_FILENAME = 'PRiFARED_maddpg_dataset_5000.csv'

# Define the precise ranges for min-max scaling later
RANGES = {
    'C_cpu': (10, 100),
    'C_data': (5, 50),
    'T_max': (100, 500),
    'Rn': (0, 9), # Integer range
    'Battery Level (%)': (1, 100),
    'Q_k': (0, 50),
    'F_max': (1, 5),
    'B_max': (5, 30),
    'P_max': (1, 6),
    'x_hat_alloc': (0, 1), # Allocations are fractions [0, 1]
    # Placeholders for results
    'Latency': (50, 600),
    'Energy': (0.5, 12.0)
}

# Define categorical mappings
TASK_CLASSES = {0: '0 (Normal)', 1: '1 (Medium)', 2: '2 (Urgent)'}
USER_CONTRACTS = {0: '0 (Normal)', 1: '1 (Premium)'}

# --- 2. HELPER FUNCTIONS ---

def generate_continuous(min_val, max_val, num_samples):
    """Generates random continuous values within a specified range."""
    return np.random.uniform(min_val, max_val, num_samples)

def generate_integer(min_val, max_val, num_samples):
    """Generates random integer values within a specified range."""
    return np.random.randint(min_val, max_val + 1, num_samples)

def generate_vector(min_val, max_val, num_samples, vector_size):
    """Generates vectors (lists of floats) for device parameters."""
    data = []
    for _ in range(num_samples):
        # Generate random values for the vector
        vector = np.random.uniform(min_val, max_val, vector_size)
        # Store as a string representation to match the original markdown format
        data.append('$$' + ', '.join(f'{x:.2f}' for x in vector) + '$$')
    return data

def generate_allocation_vector(num_samples, vector_size):
    """Generates a normalized allocation vector (sums to 1.0)."""
    data = []
    for _ in range(num_samples):
        # Generate raw random values
        raw_allocs = np.random.uniform(0.01, 1.0, vector_size)
        # Normalize so they sum to 1.0 (to respect resource limits)
        normalized_allocs = raw_allocs / np.sum(raw_allocs)
        # Store as a string
        data.append('$$' + ', '.join(f'{x:.3f}' for x in normalized_allocs) + '$$')
    return data

# --- 3. DATA GENERATION ---

data = pd.DataFrame()

# 1. Task Feature Inputs
data['Task ID'] = np.arange(1, NUM_ROWS + 1)
data['C_cpu (GC) [10-100]'] = generate_continuous(*RANGES['C_cpu'], NUM_ROWS)
data['C_data (Mbits) [5-50]'] = generate_continuous(*RANGES['C_data'], NUM_ROWS)
data['T_max (ms) [100-500]'] = generate_continuous(*RANGES['T_max'], NUM_ROWS)
data['Rn (Retry) [0-9]'] = generate_integer(*RANGES['Rn'], NUM_ROWS)
data['Battery Level (%) [1-100]'] = generate_continuous(*RANGES['Battery Level (%)'], NUM_ROWS)

# 2. Categorical Inputs (using mappings)
task_classes_raw = generate_integer(0, 2, NUM_ROWS)
user_contracts_raw = generate_integer(0, 1, NUM_ROWS)
data['Task_Service_Class [0/1/2]'] = [TASK_CLASSES[c] for c in task_classes_raw]
data['User_Contract_Type [0/1]'] = [USER_CONTRACTS[c] for c in user_contracts_raw]

# 3. Environment Vector Inputs (4 devices)
data['Q_k (Queue) [0-50]'] = generate_vector(*RANGES['Q_k'], NUM_ROWS, NUM_DEVICES)
data['F_max (GHz) [1-5]'] = generate_vector(*RANGES['F_max'], NUM_ROWS, NUM_DEVICES)
data['B_max (MHz) [5-30]'] = generate_vector(*RANGES['B_max'], NUM_ROWS, NUM_DEVICES)
data['P_max (W) [1-6]'] = generate_vector(*RANGES['P_max'], NUM_ROWS, NUM_DEVICES)

# 4. Action Placeholder Outputs (Normalized allocations)
data['x_hat_cpu (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)
data['x_hat_bw (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)
data['x_hat_pow (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)

# 5. Result Placeholder Outputs (These are synthetic results for a dataset)
data['Latency (ms) [Result]'] = generate_continuous(*RANGES['Latency'], NUM_ROWS)
data['Energy (J) [Result]'] = generate_continuous(*RANGES['Energy'], NUM_ROWS)
# Success is 1 if Latency is less than T_max, 0 otherwise (approximated for this placeholder data)
data['Success (0/1) [Result]'] = (data['Latency (ms) [Result]'] < data['T_max (ms) [100-500]']).astype(int)

# --- 4. OUTPUT AND DISPLAY ---

# Save the dataset to a CSV file (typical for Colab or training environments)
data.to_csv(OUTPUT_FILENAME, index=False)

print(f"Successfully generated {NUM_ROWS} rows of data.")
print(f"Data saved to {OUTPUT_FILENAME}")
print("\nFirst 5 rows:")
print(data.head())
print(f"\nDataFrame shape: {data.shape}")


Successfully generated 5000 rows of data.
Data saved to PRiFARED_maddpg_dataset_5000.csv

First 5 rows:
      Task ID  C_cpu (GC) [10-100]  C_data (Mbits) [5-50]  \
4995     4996            81.915540              31.767100   
4996     4997            51.768965              47.880368   
4997     4998            26.225066              43.739730   
4998     4999            81.452700              31.965853   
4999     5000            27.157884              47.111709   

      T_max (ms) [100-500]  Rn (Retry) [0-9]  Battery Level (%) [1-100]  \
4995            215.571882                 1                  23.056043   
4996            117.943995                 3                   8.714102   
4997            424.559550                 7                  32.912945   
4998            141.439951                 3                   7.498079   
4999            134.204876                 8                  94.395525   

     Task_Service_Class [0/1/2] User_Contract_Type [0/1]  \
4995              

In [5]:
display(data)

,Task ID,C_cpu (GC) [10-100],C_data (Mbits) [5-50],T_max (ms) [100-500],Rn (Retry) [0-9],Battery Level (%) [1-100],Task_Service_Class [0/1/2],User_Contract_Type [0/1],Q_k (Queue) [0-50],F_max (GHz) [1-5],B_max (MHz) [5-30],P_max (W) [1-6],x_hat_cpu (Alloc) [0-1],x_hat_bw (Alloc) [0-1],x_hat_pow (Alloc) [0-1],Latency (ms) [Result],Energy (J) [Result],Success (0/1) [Result]
0,1,63.286169,15.152048,291.372733,4,35.847576,0 (Normal),1 (Premium),"$$22.95, 23.16, 27.96, 1.69, 48.22, 22.26, 29....","$$3.60, 4.76, 2.10, 3.77, 1.94, 4.69, 3.90, 4....","$$26.15, 5.84, 11.88, 10.58, 17.89, 29.56, 29....","$$2.29, 6.00, 5.93, 3.98, 3.75, 3.76, 3.46, 1....","$$0.032, 0.048, 0.055, 0.045, 0.003, 0.031, 0....","$$0.005, 0.054, 0.010, 0.019, 0.042, 0.015, 0....","$$0.025, 0.026, 0.040, 0.021, 0.048, 0.019, 0....",285.009554,5.425720,1
1,2,29.131053,28.313197,189.054902,1,43.638175,0 (Normal),0 (Normal),"$$47.21, 19.23, 10.44, 11.33, 1.78, 41.69, 12....","$$2.43, 2.62, 1.63, 2.94, 2.28, 3.02, 4.81, 2....","$$8.38, 17.22, 15.87, 19.82, 27.78, 18.98, 16....","$$2.61, 4.52, 4.13, 3.54, 2.35, 1.93, 1.47, 2....","$$0.013, 0.002, 0.042, 0.057, 0.032, 0.011, 0....","$$0.061, 0.003, 0.014, 0.002, 0.041, 0.019, 0....","$$0.055, 0.006, 0.064, 0.060, 0.015, 0.005, 0....",211.711141,3.866398,0
2,3,25.983145,20.962681,214.101176,8,85.276074,2 (Urgent),0 (Normal),"$$44.06, 18.18, 48.50, 23.06, 16.74, 48.01, 38...","$$1.29, 2.52, 2.81, 2.56, 1.13, 1.93, 2.76, 2....","$$18.72, 7.14, 23.62, 10.72, 5.18, 8.76, 24.92...","$$3.72, 3.82, 4.11, 2.41, 2.60, 4.73, 3.31, 1....","$$0.002, 0.013, 0.027, 0.062, 0.068, 0.067, 0....","$$0.035, 0.054, 0.023, 0.066, 0.037, 0.055, 0....","$$0.032, 0.032, 0.029, 0.044, 0.033, 0.037, 0....",214.306368,11.428989,0
3,4,76.881469,16.526850,152.708016,4,97.904689,1 (Medium),1 (Premium),"$$39.99, 44.17, 23.03, 43.45, 0.00, 24.08, 9.2...","$$1.40, 3.99, 2.90, 3.14, 3.41, 1.82, 3.68, 1....","$$15.50, 19.84, 18.00, 16.41, 5.02, 29.17, 6.4...","$$2.52, 2.07, 3.09, 5.06, 4.89, 1.52, 2.30, 2....","$$0.029, 0.038, 0.013, 0.020, 0.049, 0.044, 0....","$$0.018, 0.023, 0.007, 0.056, 0.050, 0.026, 0....","$$0.029, 0.058, 0.004, 0.030, 0.044, 0.009, 0....",491.824291,10.852462,0
4,5,14.866350,21.440863,330.388023,5,7.852011,0 (Normal),0 (Normal),"$$39.13, 9.07, 10.12, 14.01, 29.86, 3.65, 11.4...","$$4.13, 1.58, 1.07, 4.11, 4.37, 2.50, 4.51, 3....","$$29.60, 22.82, 18.31, 11.23, 15.05, 24.16, 29...","$$4.41, 1.60, 3.86, 4.49, 2.24, 2.39, 2.03, 1....","$$0.041, 0.007, 0.048, 0.038, 0.058, 0.042, 0....","$$0.005, 0.032, 0.055, 0.054, 0.012, 0.054, 0....","$$0.060, 0.011, 0.007, 0.059, 0.021, 0.024, 0....",412.763056,6.769694,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,81.915540,31.767100,215.571882,1,23.056043,1 (Medium),1 (Premium),"$$9.34, 18.72, 45.49, 38.20, 14.86, 36.57, 24....","$$3.95, 2.43, 4.34, 2.39, 3.90, 3.68, 1.11, 3....","$$13.91, 20.04, 12.10, 21.42, 25.44, 15.42, 8....","$$1.20, 2.06, 1.97, 2.71, 2.57, 2.61, 5.06, 1....","$$0.045, 0.025, 0.055, 0.049, 0.023, 0.067, 0....","$$0.005, 0.040, 0.046, 0.039, 0.057, 0.049, 0....","$$0.018, 0.051, 0.014, 0.045, 0.045, 0.002, 0....",376.757744,11.237984,0
4996,4997,51.768965,47.880368,117.943995,3,8.714102,1 (Medium),0 (Normal),"$$27.95, 8.01, 37.65, 8.70, 32.76, 37.09, 1.48...","$$2.28, 2.55, 3.42, 3.76, 1.42, 3.25, 3.78, 1....","$$9.06, 18.07, 16.57, 25.87, 12.24, 15.57, 19....","$$5.19, 3.88, 3.53, 5.26, 2.21, 3.22, 4.65, 1....","$$0.057, 0.031, 0.036, 0.033, 0.060, 0.032, 0....","$$0.037, 0.004, 0.067, 0.024, 0.002, 0.036, 0....","$$0.041, 0.032, 0.022, 0.025, 0.015, 0.068, 0....",396.696119,5.371803,0
4997,4998,26.225066,43.739730,424.559550,7,32.912945,2 (Urgent),0 (Normal),"$$14.74, 18.17, 20.99, 43.34, 43.73, 28.37, 38...","$$3.73, 4.22, 1.68, 1.03, 1.78, 4.33, 3.88, 2....","$$28.08, 6.20, 17.05, 10.91, 19.97, 28.08, 9.6...","$$1.96, 5.59, 1.97, 5.35, 3.64, 4.91, 5.87, 1....","$$0.031, 0.013, 0.022, 0.025, 0.026, 0.042, 0....","$$0.057, 0.0